## 1. Packages

In [ ]:
using Agents
using GLMakie
using Random
using LinearAlgebra
using Colors
using StaticArrays

## 2. Agent definition 



In [2]:
# Define the Particle agent over a continuous 2D space
@agent struct Particle(ContinuousAgent{2,Float64})
    scale::Float64                   # Visual scale/size
    color::RGB{Float64}              # Drawing color
end

## 3. Model initialization 

In [ ]:
function initialize_model(; n_particle=100,
    speed=1.0,
    whimsy=0,
    scale=0.7,
    extent=(100, 100),
    flocking=0.0,
    vision_radius=5.0)
    # Continuous 2D toroidal space
    space2d = ContinuousSpace(extent; spacing=1, periodic=true) # Periodic boundary conditions  # PARAM

    # Global model settings
    properties = Dict(
        :collisions => 0,               # Total collision count
        :whimsy => whimsy,              # Random turn parameter
        :speed => speed,                # Movement speed
        :flocking => flocking,          # Alignment strength
        :vision_radius => vision_radius # Neighbor radius
    )

    # Build the model
    model = StandardABM(
        Particle, space2d;
        properties=properties, agent_step!,
        scheduler=Schedulers.Randomly()
    )

    # Add each particle with random position, velocity, and color

    for _ in 1:n_particle
        pos = (rand() * extent[1], rand() * extent[2])
        θ = 2π * rand() # Random angle for velocity
        vel = SVector(cos(θ), sin(θ)) # Initial velocity as unit vector
        vel = vel / norm(vel)                         # Normalize to unit vector
        col = RGB(rand(), rand(), rand())             # Random RGB color
        add_agent!(pos, model, vel, scale, col)
    end

    return model
end

initialize_model (generic function with 1 method)

## 4. Dynamics - agent_step!

In [4]:
function agent_step!(particle, model)
    if model.flocking > 0
        acc_vel = SVector(0.0, 0.0)
        count_n = 0

        # Sum velocities of neighbors within vision_radius
        for nb in nearby_agents(particle, model, model.vision_radius)
            acc_vel += nb.vel
            count_n += 1
        end
        # Compute average heading if neighbors found
        if count_n > 0
            avg_vel = acc_vel / count_n                    # Mean heading
            nv = norm(avg_vel)
            if nv > 0
                dir = avg_vel / nv                         # Unit mean direction
                # Blend current vel with flock direction
                blend = (1 - model.flocking) * particle.vel .+ model.flocking * dir
                bn = norm(blend)
                if bn > 0
                    particle.vel = blend / bn              # Update heading
                end
            end
        end
    end

    Δθ = (rand() * model.whimsy - rand() * model.whimsy) * (π / 180) # Random turn in radians
    θ0 = atan(particle.vel[2], particle.vel[1])           # Current heading angle
    θ1 = θ0 + Δθ                                          # New heading angle
    particle.vel = SVector(cos(θ1), sin(θ1))              # Apply turn


    # Move particle forward
    move_agent!(particle, model, model.speed)

    # Check for collisions with nearby agents
    neighbors = collect(nearby_agents(particle, model, 1))
    if !isempty(neighbors)
        # scatter self
        θ = (π / 180) * rand(0:359)
        particle.vel = SVector(cos(θ), sin(θ))
        move_agent!(particle, model, 0.5)  # Scatter self

        # scatter all neighbors in-radius
        for nb in neighbors
            θn = (π / 180) * rand(0:359)
            nb.vel = SVector(cos(θn), sin(θn))
            move_agent!(nb, model, 0.5)
        end

        model.collisions += 1
    end

end

agent_step! (generic function with 1 method)

## 5. GUI visualization helper



In [5]:
agent_color(p::Particle) = p.color

function particle_marker(b::Particle)
    scale_factor = 1 * b.scale
    # Define triangle vertices: back-left, front, back-rightol
    particle_polygon = Makie.Polygon(Point2f[
        (-scale_factor, -scale_factor),   # Back left vertex
        (2 * scale_factor, 0),            # Front vertex (pointing forward)
        (-scale_factor, scale_factor)     # Back right vertex
    ])

    # Rotate the triangle to match particle's heading
    φ = atan(b.vel[2], b.vel[1])
    rotate_polygon(particle_polygon, φ)
end


particle_marker (generic function with 1 method)

## 6. Launch GUI & controls 

In [ ]:
params = Dict(
    :speed => 0.1:0.1:2.0,            # Movement speed            # PARAM
    :whimsy => 0.0:10.0:359.0,        # Turning range             # PARAM
    :flocking => 0.0:0.1:1.0,         # Alignment strength        # PARAM
    :vision_radius => 1.0:1.0:20.0    # Neighbor detection radius # PARAM
)

# Run GUI
fig, _ = abmexploration(
    initialize_model(; n_particle=100, # PARAM
        scale=0.7,                         # PARAM
        extent=(100, 100));                # PARAM
    agent_color=agent_color,
    agent_marker=particle_marker,
    params=params,
    mdata=[:collisions],  # Show total collisions over time
    mlabels=["Collisions"],
    enable_inspection=false           # PARAM
)
display(fig)

GLMakie.Screen(...)